In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import torch
np.set_printoptions(suppress=True)

In [2]:
from importlib import reload
import protein_transformer
from protein_transformer.losses import mse_over_angles, mse_over_angles_numpy, inverse_trig_transform
from protein_transformer.losses import angles_to_coords

In [24]:
data = torch.load("../../data/proteinnet/casp12_191101_100.pt")

In [4]:
def add_noise(angs, stdev, truncate=True, keep_zeros=False):
    noise = np.random.normal(0, stdev, angs.shape)
    new_ang = angs + noise
    if truncate:
        new_ang[new_ang > 1] = 1
        new_ang[new_ang < -1] = -1
    if keep_zeros:
        new_ang[angs == 0] = 0
    return new_ang

def rmse(a, b):
    return np.sqrt(((a-b)**2).mean())

In [5]:
a = data["train"]["ang"][0][:]
a[np.isnan(a)] = 0
np.random.seed(0)
a_noise = add_noise(a, 1, truncate=False)
np.random.seed(0)
a_noise_truncate = add_noise(a, 1, truncate=True)
print(rmse(a, a_noise))
print(rmse(a, a_noise_truncate))

a_torch, a_noise_truncate_torch = torch.tensor(a), torch.tensor(a_noise_truncate)
print(torch.sqrt(mse_over_angles(a_torch.unsqueeze(dim=0), a_noise_truncate_torch.unsqueeze(dim=0))).item())
print(np.sqrt(mse_over_angles_numpy(np.expand_dims(a, 0), np.expand_dims(a_noise_truncate, 0))))

0.9884343496134423
0.6921556701212284
0.6921556701212276
0.6921556701212284


In [54]:
i = 10
for seq, ang, crd in zip(data["train"]["seq"][i:], data["train"]["ang"][i:], data["train"]["crd"][i:]):
    print(seq)
    print(ang)
    print(crd)
    break

VVQTEERKKANQFDAPISIYEVHLGSWRRHTDNNFWLSYRELADQLVPYAKWMGFTHLELLPINEHPFDGSWGYQPTGLYAPTRRFGTRDDFRYFIDAAHAAGLNVILDWVPGHFPTDDFALAEFDGTNLYEHSDPREGYHQDWNTLIYNYGRREVSNFLVGNALYWIERFGIDALRVDAVASMIYRDYSRKEGEWIPNEFGGRENLEAIEFLRNTNRILGEQVSGAVTMAEESTDFPGVSRPQDMGGLGFWYKWNLGWMHDTLDYMKLDPVYRQYHHDKLTFGILYNYTENFVLPLSHDEVVHGKKSILDRMPGDAWQKFANLRAYYGWMWAFPGKKLLFMGNEFAQGREWNHDASLDWHLLEGGDNWHHGVQRLVRDLNLTYRHHKAMHELDFD
[[        nan         nan -0.61309576 ...         nan         nan
          nan]
 [ 0.04925986 -0.998786   -0.56961669 ...         nan         nan
          nan]
 [ 0.2668489  -0.96373838 -0.51494968 ...         nan         nan
          nan]
 ...
 [ 0.46206156 -0.88684785  0.84435623 ...         nan         nan
          nan]
 [-0.36958297 -0.92919773  0.94677728 ...         nan         nan
          nan]
 [-0.55308486 -0.83312492         nan ...         nan         nan
          nan]]
[[59.071 85.566  2.54 ]
 [59.864 84.713  3.412]
 [59.586 83.248  3.067]
 ...
 [   nan    nan    nan]
 [   nan  

In [ ]:
def angles_have_missing_res(angles):
    return np.isnan(ang).all(axis=-1).any()

In [8]:
def get_coordinates_from_numpy_data(seq, ang_sincos):
    # Add batch dimension
    ang_sincos = ang_sincos[np.newaxis, :]
    
    # Compute angles in radians from sin/cos representaion
    ang_rad = inverse_trig_transform(torch.tensor(ang_sincos, dtype=torch.float))[0]
    
    # Remove nans
    ang_rad[torch.isnan(ang_rad)] = 0
    
    if torch.isnan(ang_rad).any():
        print("Nan in ang_rad.")
        
    seq_as_ints = protein_transformer.dataset.VOCAB.aa_seq2indices(seq, add_sos_eos=False)
    seq_as_ints = torch.tensor(seq_as_ints, dtype=torch.long)
    
    coords = angles_to_coords(ang_rad, seq_as_ints, remove_batch_padding=False)
    return coords
    
    

In [9]:
def add_noise_and_get_rmse_drmsd(seq, ang, crd, stdev):
    
    noisy_ang = add_noise(ang, stdev, truncate=True, keep_zeros=False)
#     print(noisy_ang -ang)
    
    coords = get_coordinates_from_numpy_data(seq, ang)
    noisy_coords = get_coordinates_from_numpy_data(seq, noisy_ang)
    
    rmse_val = np.sqrt(mse_over_angles_numpy(noisy_ang[np.newaxis, :], ang[np.newaxis, :]))
    drmsd_val = protein_transformer.losses.drmsd(coords, noisy_coords)
    
    crd[np.isnan(crd)] = 0
    crd[(coords == 0).all(dim=1)] = 0
    reconstruction_drmsd = protein_transformer.losses.drmsd(coords, torch.tensor(crd))
    self_drmsd_1 = protein_transformer.losses.drmsd(coords, coords)
    self_drmsd_2 = protein_transformer.losses.drmsd(torch.tensor(crd), torch.tensor(crd))
    
    return rmse_val, drmsd_val, reconstruction_drmsd, coords, self_drmsd_1, self_drmsd_2
    
    

In [10]:
r, d, d_recon, c, sd1, sd2 = add_noise_and_get_rmse_drmsd(seq, ang, crd, 1)
print(f"RMSD:\t\t{r:.4f},\nDRMSD:\t\t{d:.2f},\nDRMSD-rec:\t{d_recon:.2f},\nSelf-d1:\t{sd1},\nSelf-d2:\t{sd2}")

/home/jok120/anaconda3/envs/psp/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in greater
  """
/home/jok120/anaconda3/envs/psp/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in less
  


RMSD:		0.6890,
DRMSD:		7.49,
DRMSD-rec:	24.47,
Self-d1:	0.0,
Self-d2:	0.0


In [ ]:
c

In [11]:
from protein_transformer.protein.PDB_Creator import PDB_Creator
creator1 = PDB_Creator(c.numpy(), seq=seq)
creator2 = PDB_Creator(crd, seq=seq)

In [12]:
creator1.save_pdb("reconstructed.pdb")

In [13]:
creator2.save_pdb("original.pdb")

In [ ]:
pwd1 = protein_transformer.losses.pairwise_internal_dist(c)
pwd2 = protein_transformer.losses.pairwise_internal_dist(torch.tensor(crd))

In [ ]:
pwd2

In [ ]:
pwd1

In [ ]:
c.numpy()[:50]

In [ ]:
crd[:50]

In [ ]:
torch.isnan(c).any(), np.isnan(crd).any() 

In [ ]:
ang

In [ ]:
np.random.seed(0)
ang_noisy = add_noise(ang, 1, truncate=True, keep_zeros=True)

In [ ]:
ang_noisy = add_noise

In [ ]:
c = get_coordinates_from_numpy_data(seq, ang)

In [ ]:
c.shape

In [ ]:
ang.shape

In [ ]:
327*13